### Génération de la population de départ pour l'algorithme génétique 
>Aléatoire pour le moment

In [125]:
import random


def generate_population(Matrix, population_size, seed=22):
    
    random.seed(seed)
    
    nb_machines=len(Matrix)
    nb_task=len(Matrix[0])
    
    population=[]
    
    for i in range(population_size):
        individual=[]
        for task in range(nb_task):
            rdm_machine=random.randint(0,nb_machines-1)
            individual.append(rdm_machine)
            
        population.append(individual)
    
    return population

### Calcul du score pour un individu de la population
> C'est la durée totale que mettent tous les jobs à s'éxécuter sur les machines indiquées

In [126]:
def score_individual(Matrix, individual):
    nb_machines=len(Matrix)
    score_machine= [0 for i in range(nb_machines)]
    
    for i in range(len(individual)):
        score_machine[individual[i]]+=Matrix[individual[i]][i]
        
    return max(score_machine)

### Récupération de 50% de la population actuelle
>45% sont ceux ayant le meilleur score

>5% sont séléctionnés aléatoirement

In [127]:
def selection_fitness(Matrix, population):
    sorted_pop=sorted(population, key=lambda individual: score_individual(Matrix, individual))
    
    nb_fittest=int(len(population)*0.45)
    nb_random=int(len(population)*0.5)-nb_fittest
    
    parents=sorted_pop[0:nb_fittest]
    
    for i in range(nb_random):
        rdm=random.randint(nb_fittest+1, len(population)-1)
        parents.append(sorted_pop[rdm])
    
    return list(parents)
    
for i in selection_fitness(M, population):
    print(score_individual(M, i))

5
6
6
6
6
6
6
6
6
10


### Crossover
> Les parents font un accouplement 2 à 2 (Chaleeeuuur)

> L'enfant généré récupère une partie de chacun de ses parents (la taille de chaque partie étant définie aléatoirement)

In [128]:
def crossover(parents):
    
    children=[]
    for i in range(len(parents)):
        p1=parents[i]
        if (i!=len(parents)-1):
            p2=parents[i+1]
        else :
            p2=parents[0]
        
        rdm=random.randint(1,len(p1)-2)
        child = p1[0:rdm] + p2[rdm:]
        
        children.append(child)

    return list(children)

#crossover(selection_fitness(M, population))
        

### Mutations
> 5% de la nouvelle population (parents+enfants) est mutée

> Les mutés subissent un changement sur l'une des valeurs du vecteur (un des jobs ne s'exécute plus sur la même machine)

In [129]:
def mutator(Matrix, population):
    nb_mutated=int(len(population)*0.05)
    for i in range(nb_mutated):
        rdm_individual=random.randint(0, len(population)-1)
        rdm_job=random.randint(0,len(Matrix[0])-1)
        rdm_machine=random.randint(0, len(Matrix)-1)
        
        population[rdm_individual][rdm_job]=rdm_machine
        
    return list(population)

### The fittest
> Pour le résultat à la fin de l'algorithme général

In [135]:
def the_fittest(Matrix, population):
    sorted_pop=sorted(population, key=lambda individual: score_individual(Matrix, individual))
    
    return sorted_pop[0], score_individual(Matrix, sorted_pop[0])

## Algorithme génétique (sisi)

In [141]:
def genetic_algo(Matrix, population_size):
    population = generate_population(Matrix, population_size)
    for i in range(1000):
        parents = selection_fitness(Matrix, population)
        children = crossover(parents)
        population = mutator(Matrix, parents+children)
    
    return the_fittest(Matrix, population)   

# TEST

In [142]:
M= [[1,2,3,4,5,6,7,8,9],[9,8,7,6,5,4,3,2,1],[2,3,9,8,7,5,1,4,2],
    [7,4,1,8,5,2,9,6,3],[3,6,9,2,5,8,1,4,7],[6,5,4,3,2,1,9,8,7]]
print(genetic_algo(M,100))

([0, 0, 3, 4, 5, 5, 2, 1, 1], 3)
